In [4]:
import sys
import torch
import torch.nn as nn
import pennylane as qml
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"Pennylane version: {qml.__version__}")


def weird_embedding(x):
    # x is supposed to be a single data, not a batch of data
    print(f"x.shape = {x.shape}")
    for i in range(x.shape[1]-1):
        qml.CRX(x[:, i], wires=[i, (i+1)])
        qml.T(wires=i)
        qml.CRY(x[:, i], wires=[i, (i+1)])
        qml.S(wires=(i+1))
        qml.CRZ(x[:, i], wires=[i, (i+1)])
    qml.Hadamard(wires=0)

device = qml.device("default.qubit", wires=3)
@qml.qnode(device)
def circuit(inputs, weights):
    print(f"circuit inputs shape = {inputs.shape}")
    # qml.AngleEmbedding(inputs, wires=range(2))
    # qml.AmplitudeEmbedding(inputs, wires=range(2), pad_with=0, normalize=True)
    # qml.IQPEmbedding(inputs, wires=range(2))
    weird_embedding(inputs)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=(0,1))
    return qml.expval(qml.PauliX(0))

torch_layer = qml.qnn.TorchLayer(circuit, weight_shapes = {"weights": (0)})
net = nn.Sequential(torch_layer)

batch_size = (5,4,3)
inputs_shape = (2)
dataset = torch.rand(*batch_size, inputs_shape)
print(f"net(dataset) shape = {net(dataset).shape}")

Python version: 3.9.12 (main, May 25 2023, 22:45:03) 
[GCC 9.4.0]
PyTorch version: 2.0.1+cu118
Pennylane version: 0.33.1
circuit inputs shape = torch.Size([60, 2])
x.shape = torch.Size([60, 2])
net(dataset) shape = torch.Size([5, 4, 3])
